<a href="https://colab.research.google.com/github/cedjustin/AI/blob/master/sprint13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [235]:
# import dependencies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
tf = tf.compat.v1

[Problem 1] Looking back on the scratch
* had an epoch loop
* had to initialize weight and bias
* using mini batches
* have to calculate forward propagation and backward propagation
* using activation functions





[Problem 2] Consider the correspondence between scratch and TensorFlow

Using the numpy scratch vs tensorflow, tensorflow reduces the time of implementation and it gives more complexity to the things that you are implementing and which also results in giving you more control on what you are doing

【Problem 3】Create a model of Iris using all three object variables

In [236]:
class GetMiniBatch:
    """
    Iterator to get a mini-batch
    Parameters
    ----------
    X : The following forms of ndarray, shape (n_samples, n_features)
      Training data
    y : The following form of ndarray, shape (n_samples, 1)
      Correct answer value
    batch_size : int
      Batch size
    seed : int
      NumPy random number seed
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [256]:
def estimation_calc(X, Y, X_train, y_train, X_val, y_val, X_test, y_test, num_epochs, get_mini_batch_train):
  def example_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    # Declaration of weight and bias
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
    return layer_output
  #Read network structure                              
  logits = example_net(X)
  # Objective function
  loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
  # Optimization method
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
  train_op = optimizer.minimize(loss_op)
  # Estimated result
  correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.nn.softmax(logits) - 0.5))
  #Indicator value calculation
  accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
  #Initialization of variable
  init = tf.global_variables_initializer()

  #Run calculation graph
  with tf.Session() as sess:
      sess.run(init)
      for epoch in range(num_epochs):
          #Loop for each epoch
          total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
          total_loss = 0
          total_acc = 0
          for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
              # Loop for each mini-batch
              sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
              loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
              total_loss += loss
          total_loss /= n_samples
          val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
          # print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
      test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
      print("test_acc : {:.3f}".format(test_acc))

Calculation using 3 species

In [257]:
#Load dataset
df = pd.read_csv("Iris.csv")
#Condition extraction from data frame
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
# NumPy 配列に変換
X = np.array(X)
y = np.array(y)
# Convert label to number
y[y == "Iris-setosa"] = 0
y[y == "Iris-versicolor"] = 1
y[y == "Iris-virginica"] = 2
y = y.astype(np.int64)[:, np.newaxis]

#Split into train and test
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train2, X_val2, y_train2, y_val2 = train_test_split(X_train2, y_train2, test_size=0.2, random_state=0)

# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train2.shape[1]
n_samples = X_train2.shape[0]
n_classes = 1
#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# train mini batch iterator
get_mini_batch_train2 = GetMiniBatch(X_train2, y_train2, batch_size=batch_size)
estimation_calc(X, Y, X_train2, y_train2, X_val2, y_val2, X_test2, y_test2, num_epochs, get_mini_batch_train2)

test_acc : 0.633


Calculation using house prices